In [1]:
import cv2
import numpy as np
import os


part 1

In [2]:
img1 = cv2.imread("C:/sem 6/cv/assignment3/panaroma_generation/1.jpg")
img2 = cv2.imread("C:/sem 6/cv/assignment3/panaroma_generation/2.jpg")

In [3]:
sift = cv2.SIFT_create()


In [4]:
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)


In [5]:
cv2.drawKeypoints(img1, kp1, img1, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.drawKeypoints(img2, kp2, img2, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


array([[[196, 193, 185],
        [206, 203, 195],
        [233, 230, 222],
        ...,
        [219, 217, 209],
        [221, 219, 211],
        [220, 218, 210]],

       [[207, 204, 196],
        [195, 192, 184],
        [228, 225, 217],
        ...,
        [219, 217, 209],
        [218, 216, 208],
        [218, 216, 208]],

       [[206, 203, 195],
        [189, 202, 178],
        [133, 244, 127],
        ...,
        [218, 216, 208],
        [217, 215, 207],
        [218, 216, 208]],

       ...,

       [[ 50,  95,  76],
        [ 41,  86,  67],
        [ 39,  82,  67],
        ...,
        [ 44,  84,  82],
        [ 74, 115, 110],
        [ 51,  92,  87]],

       [[ 41,  83,  66],
        [ 41,  83,  66],
        [ 14,  55,  40],
        ...,
        [ 44,  84,  82],
        [ 69, 109, 107],
        [ 46,  86,  84]],

       [[ 25,  67,  50],
        [  6,  48,  31],
        [ 50,  90,  78],
        ...,
        [ 47,  87,  85],
        [ 48,  88,  86],
        [ 52,  92,  90]]

In [6]:
cv2.imshow("Image 1 with Keypoints", img1)
cv2.imshow("Image 2 with Keypoints", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


part 2

In [7]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
matches_bf = bf.knnMatch(des1, des2, k=2)


In [8]:
flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
matches_flann = flann.knnMatch(des1, des2, k=2)


In [9]:
good_matches_bf = []
good_matches_flann = []
img_matches_bf = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches_bf, None, flags=2)
img_matches_flann = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches_flann, None, flags=2)

for m, n in matches_bf:
    if m.distance < 0.7*n.distance:
        good_matches_bf.append(m)

for m, n in matches_flann:
    if m.distance < 0.7*n.distance:
        good_matches_flann.append(m)

img_matches_bf = cv2.drawMatches(img1, kp1, img2, kp2, good_matches_bf, img_matches_bf, flags=2)
img_matches_flann = cv2.drawMatches(img1, kp1, img2, kp2, good_matches_flann, img_matches_flann, flags=2)

cv2.imshow("Matches - Brute-Force", img_matches_bf)
cv2.imshow("Matches - FlannBased", img_matches_flann)
cv2.waitKey(0)
cv2.destroyAllWindows()


part 3

In [10]:
def find_homography(kp1, kp2, des1, des2):

  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
  matches = bf.knnMatch(des1, des2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.7*n.distance:
          good_matches.append(m)

  src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
  dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

  M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
  return M


M = find_homography(kp1, kp2, des1, des2)
print(M)



[[-6.39038505e+01  1.88277579e+00  2.30121106e+04]
 [-1.93576646e+01 -3.90627778e+01  1.19679960e+04]
 [-6.64474794e-02 -4.94887013e-04  1.00000000e+00]]


part 4

In [11]:
res1 = cv2.warpPerspective(img1, M, (img1.shape[1], img1.shape[0]))
res2 = cv2.warpPerspective(img2, M, (img2.shape[1], img2.shape[0]))
combined_img = np.concatenate((res1, img2), axis=1)
cv2.imshow("Warped Images", combined_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


part 5

In [12]:

combined_img = np.concatenate((res1, img2), axis=1)
cv2.imshow("Warped Images without blending", combined_img)

alpha = 0.5  # blending factor
blended_image = cv2.addWeighted(res1, alpha, img2, 1 - alpha, 0)
cv2.imshow("Warped Images with blending", blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

part 6

In [13]:
folder_path = "C:/sem 6/cv/assignment3/panaroma_generation"

image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
M_list = []
images = []
matches_list = []
blended_images=[]

for path in image_paths:
    temp = cv2.imread(path)
    images.append(temp)

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

for i in range(len(images)-1):
    img1 = images[i]
    img2 = images[i+1]
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    matches = bf.knnMatch(des1, des2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matches_list.append(good_matches)
    M_list.append(M)

    res = cv2.warpPerspective(img1, M, (img1.shape[1], img1.shape[0]))
    res2 = cv2.warpPerspective(img2, M, (img2.shape[1], img2.shape[0]))
    alpha = 0.5  # blending factor
    blended_image = cv2.addWeighted(res1, alpha, img2, 1 - alpha, 0)
    blended_images.append(blended_image)

stitcher = cv2.Stitcher_create()
    
status, stitched_image = stitcher.stitch(images)
 
if status == cv2.Stitcher_OK:

    cv2.imshow('Stitched Image', stitched_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Stitching failed!")

# combined_img = np.concatenate(tuple(blended_images), axis=1)
# cv2.imshow("Warped Images", combined_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [64]:
print(len(blended_images))

7
